<a href="https://colab.research.google.com/github/ealder04/ME485IMUProject/blob/master/nnTut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Processing the data. Data normalization could be helpful.

In [ ]:
dataframe = pandas.read_csv('https://github.com/ealder04/ME485IMUProject/raw/acb203bc63e3e2e40ad013d246a26e23003f0a40/M_Stroke', delim_whitespace=True, header=None)
Y = dataframe.values

dataframe = pandas.read_csv('https://github.com/ealder04/ME485IMUProject/raw/acb203bc63e3e2e40ad013d246a26e23003f0a40/StrokeStateFile', delim_whitespace=True, header=None)
X = dataframe.values

X0 = X[:,109:]
X = X[:,0:14]

X = np.concatenate((X,X0), axis=1)


In [ ]:
seed = 7
np.random.seed(seed)
CONTINUE_TRAINING = False

if not CONTINUE_TRAINING:
  model = Sequential()
  model.add(Dense(273, input_dim=17, kernel_initializer='normal', activation='relu'))
  model.add(Dense(50, kernel_initializer='normal', activation='relu'))
  model.add(Dense(47, kernel_initializer='normal'))

	# Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
else:
	model = load_model('nnregression.h5')

X, Y = shuffle_in_unison(X,Y)

In [ ]:
split_size = int(len(X) * 0.8)
X_train, X_val = X[:split_size], X[split_size:]
Y_train, Y_val = Y[:split_size], Y[split_size:]
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
model.fit(X_train,Y_train, epochs=1000, batch_size=10, verbose=2, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Epoch 1/1000
771/771 - 2s - loss: 0.3687 - val_loss: 0.3428
Epoch 2/1000
771/771 - 1s - loss: 0.3325 - val_loss: 0.3180
Epoch 3/1000
771/771 - 1s - loss: 0.3087 - val_loss: 0.2973
Epoch 4/1000
771/771 - 1s - loss: 0.2900 - val_loss: 0.2862
Epoch 5/1000
771/771 - 1s - loss: 0.2758 - val_loss: 0.2713
Epoch 6/1000
771/771 - 1s - loss: 0.2627 - val_loss: 0.2545
Epoch 7/1000
771/771 - 1s - loss: 0.2533 - val_loss: 0.2503
Epoch 8/1000
771/771 - 1s - loss: 0.2455 - val_loss: 0.2414
Epoch 9/1000
771/771 - 1s - loss: 0.2376 - val_loss: 0.2363
Epoch 10/1000
771/771 - 1s - loss: 0.2313 - val_loss: 0.2275
Epoch 11/1000
771/771 - 1s - loss: 0.2268 - val_loss: 0.2207
Epoch 12/1000
771/771 - 1s - loss: 0.2216 - val_loss: 0.2224
Epoch 13/1000
771/771 - 1s - loss: 0.2167 - val_loss: 0.2163
Epoch 14/1000
771/771 - 1s - loss: 0.2124 - val_loss: 0.2161
Epoch 15/1000
771/771 - 1s - loss: 0.2101 - val_loss: 0.2070
Epoch 16/1000
771/771 - 1s - loss: 0.2057 - val_loss: 0.2047
Epoch 17/1000
771/771 - 1s - loss

The model should be evaluated on the test set...here we are comparing the prediction on the validation set.

In [ ]:
predictions = model.predict(X_val)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(predictions[1,:]) 
print(Y_val[1,:])

[0.800 -0.191 -0.070 -0.190 -0.043 0.760 -0.160 0.480 0.020 0.074 -0.392
 -0.292 -0.574 -0.752 -0.313 0.508 -0.705 -0.050 -0.125 -0.236 0.675
 -0.707 0.533 0.233 -0.435 -0.719 -0.765 -0.140 -0.140 -1.059 0.581 -0.458
 0.115 -0.033 -0.842 0.067 -0.173 0.095 -0.004 0.414 -0.636 0.215 0.714
 0.398 0.533 -0.613 0.469]
[0.813 -0.179 -0.000 -0.378 -0.051 0.673 -0.116 0.443 -0.004 0.083 -0.410
 -0.297 -0.736 -1.247 -0.519 0.839 -1.052 -0.037 -0.295 -0.163 0.913
 -1.021 0.919 0.255 -0.736 -1.048 -1.177 -0.140 -0.021 -1.040 0.640 -0.953
 0.235 -0.083 -1.127 -0.080 -0.470 0.296 -0.115 0.604 -0.878 0.381 1.064
 0.629 0.626 -0.755 0.485]
